# Projeto 1 - Ciência dos Dados

Nome: Jonas Bonfá Pelegrina

Nome: Ricardo Mourão Rodrigues Filho

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

## Introdução:

O Nosso trabalho consiste em um tema muito saboroso dependendo aos olhos de quem vê, ou melhor, de quem saboreia! O tópico que desejamos trabalhar é o "Big Mac", o famoso sanduíche do "McDonalds". Mas será que as pessoas possuem um sentimento positvo em relação a ele?

Com isso, vamos analisar "tweets" de usuários do Twitter avaliando o quanto aquela informação é relevante para nossa classificação. 

Então qual é o critério para uma classifição relevante?

- Se o tweet possui um sentimento positivo em relação ao "big Mac", ele recebe uma classificação "1";
- Caso não, ele recebe uma classificação "0".

Ademais, usamos o *Classificador Naive-Bayes* para nos ajudar nos cálculos matématicos.

<img src="https://veja.abril.com.br/wp-content/uploads/2016/07/big-mac-mc-donalds.jpg?quality=70&strip=info&resize=680,453" alt="foto do bigmac">

___
Carregando algumas bibliotecas:

In [28]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display
pd.options.display.max_rows = 13

In [29]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/jonaspelegrina/OneDrive - Insper - Institudo de Ensino e Pesquisa/Ciência dos dados/Projeto01_etapa2/P1


___
## Montando um Classificador Naive-Bayes:

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador. Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [30]:
filename = 'big mac.xlsx'

In [31]:
train = pd.read_excel(filename)
train['Classificação']=train['Unnamed: 1']
del train['Unnamed: 1']
train

,Treinamento,Classificação
0,@srtaa_reisp ainda me realizo com esse big mac,1
1,@nxx0hx @g1 excesso de big mac e coca-cola no ...,1
2,acordei com um big mac me esperando na mesa \n...,1
3,bateu vontade de comer um big mac,1
4,rt @fee_araaujo: queria almoçar um big mac 😭,1
...,...,...
295,quem usa a frase “tem pão com ovo se achando b...,0
296,1 fav e eu peço um big mac,0
297,rt @carloscalmeida3: fala q é socialista/comun...,0
298,@spampijoana ao menos contigo estou descansado...,0


### Removendo informações desnecessárias:

In [32]:
import re 

# Função que remove pontuação
def cleanup(texto):
    pontuacao = '[\/!-.:?;]'
    padronização = re.compile(pontuacao)
    texto_substituido = re.sub(padronização, '', texto)
    return texto_substituido

for i in range(300): 
    # Usamos o método replace() para substituir uma frase especificada por outra frase especificada.
    # Removendo pontuação com o replace
    train = train.replace(train['Treinamento'][i], cleanup(train['Treinamento'][i]))
    
    # Removendo 'rt' com replace
    if train['Treinamento'][i][:2] == 'rt':
        train = train.replace(train['Treinamento'][i], train['Treinamento'][i][3:])
    
    # Removendo '\n' com o replace
    train['Treinamento'] = train['Treinamento'].str.replace('[\n]','')
    
    # Deixando todas as letras minúsculas com replace:
    # O método lower() retorna uma string em que todos os caracteres são minúsculos
    train = train.replace(train['Treinamento'][i], train['Treinamento'][i].lower()) 

In [33]:
train

,Treinamento,Classificação
0,@srtaa_reisp ainda me realizo com esse big mac,1
1,@nxx0hx @g1 excesso de big mac e cocacola no c...,1
2,acordei com um big mac me esperando na mesa ai...,1
3,bateu vontade de comer um big mac,1
4,@fee_araaujo queria almoçar um big mac 😭,1
...,...,...
295,quem usa a frase “tem pão com ovo se achando b...,0
296,1 fav e eu peço um big mac,0
297,@carloscalmeida3 fala q é socialistacomunista ...,0
298,@spampijoana ao menos contigo estou descansado...,0


### *Tweets* relavantes

In [34]:
# tweets relevantes
tweets_1 = train.loc[(train['Classificação'] == 1)]
tweets_1.head()

,Treinamento,Classificação
0,@srtaa_reisp ainda me realizo com esse big mac,1
1,@nxx0hx @g1 excesso de big mac e cocacola no c...,1
2,acordei com um big mac me esperando na mesa ai...,1
3,bateu vontade de comer um big mac,1
4,@fee_araaujo queria almoçar um big mac 😭,1


### *Tweets* irrelevantes:

In [35]:
# tweets irrelevantes
tweets_0 = train.loc[(train['Classificação'] == 0)]
tweets_0.head()

,Treinamento,Classificação
150,vem comer seu big mac sem queijo e sem maiones...,0
151,@ruizeto_big_mac bruh ahahahahahahahahahahah t...,0
152,fala q é socialistacomunista ataca o capitalis...,0
153,pedi big mac e matei minha vontade credo,0
154,@andrey1980 siiiiim eu nem sabia o que era big...,0


### Probabilidade do *tweet* ser relevante (P_1):

In [36]:
P_1 = tweets_1.Treinamento.count()/train.Treinamento.count()
print('A probabilidade dos tweets relevantes é: {0}' .format(P_1))

A probabilidade dos tweets relevantes é: 0.5


### Probabilidade do *tweet* ser irrelevante (P_0):

In [37]:
P_0 = tweets_0.Treinamento.count()/train.Treinamento.count()
print('A probabilidade dos tweets irrelevantes é: {0}' .format(P_0))

A probabilidade dos tweets irrelevantes é: 0.5


### Frequência absoluta das palavras que aparecem nos *tweets* relevantes:

In [38]:
# Formando um texto com todas as palavras relevantes nele:
texto_1 = " ".join(tweets_1['Treinamento'])

#transformando o texto armazenado, em um pd.series usando a função split(), quebrando cada palavra do texto:
series_1 = pd.Series(texto_1.split())
#Contabilizando as palavras no texto relevante:
series_1 = series_1.value_counts()
print('O número de palavras totais no grupo de relevantes é: {0}'.format(series_1.sum()))
print('O número de palavras totais, não repetidas, no grupo de relevantes é: {0}'.format(len(series_1)))

O número de palavras totais no grupo de relevantes é: 1799
O número de palavras totais, não repetidas, no grupo de relevantes é: 621


### Frequência absoluta das palavras que aparecem nos *tweets* irrelevantes:

In [39]:
# Formando um texto com todas as palavras relevantes nele:
texto_0 = " ".join(tweets_0['Treinamento'])

#transformando o texto armazenado em uma série usando a função split(), quebrando cada palavra do texto:
series_0 = pd.Series(texto_0.split())
#Contabilizando as palavras no texto irrelevante:
series_0 = series_0.value_counts()
print('O número de palavras totais no grupo de irrelevantes é: {0}'.format(series_0.sum()))
print('O número de palavras totais, não repetidas, no grupo de irrelevantes é: {0}'.format(len(series_0)))

O número de palavras totais no grupo de irrelevantes é: 2367
O número de palavras totais, não repetidas, no grupo de irrelevantes é: 903


### Frequência absoluta total:

In [40]:
# Formando um texto total com todas as palavras nele:
texto_total = texto_1 + texto_0
#transformando o texto armazenado em uma série usando a função split(), quebrando cada palavra do texto:
series = pd.Series(texto_total.split())
#Contabilizando as palavras no texto total:
series = series.value_counts()
print('O número de palavras totais é: {0}'.format(series.sum()+1))
print('O número de palavras totais, não repetidas, é: {0}'.format(len(series)))

O número de palavras totais é: 4166
O número de palavras totais, não repetidas, é: 1281


### Que conclusões podemos tomar de acordo com as frequências absolutas calculadas?  <br>
<ul>
    <li>A probabilidade de uma palavra ser relevante $P(1)$ pode ser achada;</li>
    <li>A probabilidade de uma palavra ser irrelevante $P(0)$ pode ser achada.</li>
</ul>

In [41]:
# Achando P(1)
P_palavra_1 = len(texto_1.split())/len(texto_total.split())
P_palavra_1

0.4319327731092437

In [42]:
# Achando P(0)
P_palavra_0 = len(texto_0.split())/len(texto_total.split())
P_palavra_0

0.5683073229291716

___
## Verificando a performance do Classificador:

Agora você deve testar o seu classificador com a base de Testes. Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [43]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test['Classificação']=test['Unnamed: 1']
del test['Unnamed: 1']
test

,Teste,Classificação
0,tudo q eu queria agr é ta todo encolhido na sa...,1
1,"queria um big mac, uma batata grande e um milk...",1
2,@its_lela @burgerkingbr desmereceram meu big m...,1
3,achei um big mac na geladeira que salvou 🤣,1
4,hoje minha família inteira jantou big mac. tá ...,1
...,...,...
195,já não comia um big mac há algum tempo,0
196,"o big mac r$0,99 mas nem isso eu tenho pra com...",0
197,"""errado é o febem quando foi comprar um big ma...",0
198,rt @mancheterevista: linha de frente do projot...,0


### Removendo informações desnecessárias:

In [44]:
import re 

# Função que remove pontuação
def cleanup(texto):
    pontuacao = '[\/!-.:?;]'
    padronização = re.compile(pontuacao)
    texto_substituido = re.sub(padronização, '', texto)
    return texto_substituido

for i in range(200):
    # Usamos o método replace() para substituir uma frase especificada por outra frase especificada.
    # Removendo pontuação com o replace
    test = test.replace(test['Teste'][i], cleanup(test['Teste'][i]))
    
    # Removendo 'rt' com replace
    if test['Teste'][i][:2] == 'rt':
        test = test.replace(test['Teste'][i], test['Teste'][i][3:])
    
    # Removendo '\n' com o replace
    test['Teste'] = test['Teste'].str.replace('[\n]','')
    
    # Deixando todas as letras minúsculas com replace:
    # O método lower() retorna uma string em que todos os caracteres são minúsculos
    test = test.replace(test['Teste'][i], test['Teste'][i].lower())

In [45]:
test

,Teste,Classificação
0,tudo q eu queria agr é ta todo encolhido na sa...,1
1,queria um big mac uma batata grande e um milks...,1
2,@its_lela @burgerkingbr desmereceram meu big m...,1
3,achei um big mac na geladeira que salvou 🤣,1
4,hoje minha família inteira jantou big mac tá f...,1
...,...,...
195,já não comia um big mac há algum tempo,0
196,o big mac r099 mas nem isso eu tenho pra comprar,0
197,errado é o febem quando foi comprar um big mac...,0
198,@mancheterevista linha de frente do projota p...,0


### Comparação para o Naïve-Bayes:

Para classificarmos os tweets da nossa planilha teste, vamos usar a determinada comparação:
<br>
$P(1|tweet) > P(0|tweet)$
<br>
Ou seja, a probabilidade do meu tweet ser relevante é maior que a probabilidade de ser irrelevante.
### Para que serve essa comparação?

Justamente para determinarmos futuramente uma nova coluna na base de Teste de acordo com essa comparação.

Sabemos que:<br>
$P(1|tweet) = \frac{P(tweet|1)P(1)}{P(tweet)}$
<br>
$P(0|tweet) = \frac{P(tweet|0)P(0)}{P(tweet)}$<br>
Entretanto, a Naïve-Bayes facilita os nossos cálculos, em que vamos desconsiderar o denominador, ou seja, iremos comparar apenas os numeradores.

### Suavização de Laplace para $P(1|tweet)$:

In [46]:
# Agora vamos calcular dado um tweet, a probabilidade dele ser relevante.
# Para isso, precisamos descobrir dado um espaço amostral de relevância qual a probabilidade do meu tweet estar nele?
# Inicialmente, vamos definir a variável da probabilidade de ser relevante dado um tweet igual a probabilidade de uma palavra 
# ser relevante(logo logo você irá entender):
P_1dadoT = P_palavra_1
# agora vamos definir uma lista para armazenar essas variáveis:
lista_1 = []
#analisando um tweet:
for tweet in test.Teste:
    #analisando a palavra dentro do tweet:
    for palavra in tweet.split():
        if palavra not in series_1:
            P_1dadoT *= (0 + 1) / (series_1.sum() + len(series))
        else:
            P_1dadoT *= (series_1[palavra] + 1)/(series_1.sum() + len(series))
    lista_1.append(P_1dadoT)
    P_1dadoT = P_palavra_1
len(lista_1)

200

### Suavização de Laplace para $P(0|tweet)$:

In [47]:
# Agora vamos calcular dado um tweet, a probabilidade dele ser irrelevante.
# Para isso, precisamos descobrir dado um espaço amostral de irrelevância qual a probabilidade do meu tweet estar nele?
# Inicialmente, vamos definir a variável da probabilidade de ser irrelevante dado um tweet igual a probabilidade de uma palavra 
# ser irrelevante(logo logo você irá entender):
lista_0 = []
# agora vamos definir uma lista para armazenar essas variáveis:
P_0dadoT =  P_palavra_0
#analisando um tweet:
for tweet in test.Teste:
    #analisando a palavra dentro do tweet:
    for palavra in tweet.split():
        if palavra not in series_0:
            P_0dadoT *= (0 + 1)/(series_0.sum() + len(series))
        else:
            P_0dadoT *= (series_0[palavra] + 1)/(series_0.sum() + len(series))
    lista_0.append(P_0dadoT)
    P_0dadoT = P_palavra_0
len(lista_0)

200

### Criando nova coluna de acordo com a comparação para o Naïve-Bayes:

In [48]:
comparador_Naive_Bayes = []
for i in range(len(lista_1)):
    if lista_0[i] > lista_1[i]:
        comparador_Naive_Bayes.append(0)
    else:
        comparador_Naive_Bayes.append(1)
test['Comparação'] = comparador_Naive_Bayes
test

,Teste,Classificação,Comparação
0,tudo q eu queria agr é ta todo encolhido na sa...,1,1
1,queria um big mac uma batata grande e um milks...,1,1
2,@its_lela @burgerkingbr desmereceram meu big m...,1,1
3,achei um big mac na geladeira que salvou 🤣,1,1
4,hoje minha família inteira jantou big mac tá f...,1,1
...,...,...,...
195,já não comia um big mac há algum tempo,0,1
196,o big mac r099 mas nem isso eu tenho pra comprar,0,0
197,errado é o febem quando foi comprar um big mac...,0,1
198,@mancheterevista linha de frente do projota p...,0,0


In [59]:

print("Podemos notar de acordo com nosso classificador que a maioria das pessoas possuem um sentimento positivo em relação ao bigmac: {0} pessoas das 200,possuem sentimento positivo.".format(len(test.loc[test.Comparação == 1,:])))

Podemos notar de acordo com nosso classificador que a maioria das pessoas possuem um sentimento positivo em relação ao bigmac: 125 pessoas das 200,possuem sentimento positivo.


### Acurácia:

In [49]:
exatidao = 0
for i in range(200):
    if test['Classificação'][i] == test['Comparação'][i]:
        exatidao += 1
print('Tivemos um total de {0}% de exatidão.'.format((exatidao/200)*100))

Tivemos um total de 75.5% de exatidão.


### Tabela de visualização:

In [50]:
tabela = pd.crosstab(test['Classificação'],test['Comparação'], normalize = True)*100
tabela

Comparação,0,1
Classificação,,
0,31.5,18.5
1,6.0,44.0


### Resultados:

In [60]:
print('Verdadeiros Negativos: {0}%'.format(tabela[0][0]))
print('Falsos Negativos: {0}%'.format(tabela[1][0])) 
print('Falsos Positivos: {0}%'.format(tabela[0][1].round(1)))
print('Verdadeiros Positivos: {0}%'.format(tabela[1][1]))

Verdadeiros Negativos: 31.5%
Falsos Negativos: 18.5%
Falsos Positivos: 6.0%
Verdadeiros Positivos: 44.0%


___
## Conclusão

De acordo com a acurácia, obtivemos uma exatidão de 75.5% de acertos, tomando como comparação o nosso classificador. Isso significa que apesar de diversos fatores, os integrantes do grugo apresentaram uma concepção parecida em relação ao critério de avaliação dos tweets (Um sentimento positivo ao "Big Mac").

Somando os falsos negativos com os falsos postivos chegamos a uma porcentagem baixa de 24,5% de erros(Valores que nós avaliamos diferentemento do classificador).

Vale ressaltar que essa exatidão é influenciada por outros diversos fatores que impedem uma frequência de 100% nas classificações, como por exemplo: o contexto na qual o "tweet" foi escrito, a ironia (sarcasmo),e a dupla negação,pois como de forma simplificada o que o nosso classificador faz é contar palavras,e ele não sabe se essa palavra que contou está sendo utilizada em um contexto ironico ou de dupla negação,portanto não sendo totalmente efetivo.Além de não entender o significado de cada emoji e os usuários marcados etc.

Após a nossa análise verificamos que o público realmente tem uma relação positiva com o bigmac.Podemos notar de acordo com nosso classificador que a maioria das pessoas possuem um sentimento positivo em relação ao bigmac: 125 pessoas das 200,possuem sentimento positivo.Então esse projeto deve continuar a ser investido para que essa relação de afinidade com o público continue sendo monitorada.

<h3> Por por que não posso alimentar minha base de Treinamento
automaticamente usando o próprio classificador, aplicado a novos tweets? </h3> 

Nós não podemos utilizar o nosso próprio classificador para novas amostras pois,em seu funcimento ele utiliza uma base pré existente e se utilizamos ele para alimentar o treinamento ele começaria a comparar esses novos exemplos com ele mesmo.
<h3>Diferentes cenários para o Naive-Bayes</h3>

Podemos utilizar o classificado de Naive-Bayes principalmente em análise de textos,filtragem de spam,probabilidade de hipóteses.

<h3>Sugestão de melhorias</h3>
1 . Aumentar o número de integrantes da equipe que manualmente classificam os tweets pois dessa maneira o classificador vai entender melhor o conceito de relevância e irrelevância de determinado grupo.
<br>
2 . A partir de padrões seria interessantes se o classificador notar a presença de ironia e dupla negação para que esses erros não influenciem negativamente,já que é um uso comum a ironica e a dupla negação
<br>
3.Segmentar melhor as informações para que o classificador não tenha tantos problemas na classificação pois o tema/grupo que está sendo analisado é menor e mais específico.
<br>
https://machinelearningmastery.com/better-naive-bayes/

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**